## Keras General Adversarial Networks (GANS)
This tutorial is on how to create a General Adversarial Network Based on a auto encoder to generate simulated data. This pariticular work as adapted from:

https://github.com/eriklindernoren/Keras-GAN#adversarial-autoencoder

This essentially builds a very basic GAN using only feedforward neural networks, and no complex layers


#### Components of a Gan

###### Generator
+ takes random data as input
+ predicts output that looks like our training data
+ trainable weights
+ Not Training Seperately, training inside stacked model

###### Discriminator
+ takes real and fake data combined in one array
+ trains against [0,1] fake vs real labels

###### Stacked Generator and Discriminator
+ Generator + Discriminator (with the weights set to be no trainable ie frozen)

##### Create Data Function
+ takes in real data, generator
+ returns real_data, fake_data of the same size and shape

##### Training Function
+ takes in real_data, discriminator, stacked model
+ During Each Epoc 
    + trains the distriminator on a 50/50 mix of real and fake images against [0,1] labels
    + trains stacked model with disciminator wieghts frozed on input noise, against an array of all 1s\
        + this force the generator part of the stacked model to improve it's generation ablity


In [15]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, merge
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import Adam

from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import logging
# base logger setup, to standardize logging across classes
try:
    logger.debug('testing logger')
except:
    name = 'GANs'
    formatter = logging.Formatter(fmt='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    logger.addHandler(handler)

##### Data 
For this experiment, data from sklearn's Boston Housing is used

In [2]:
# loads data 
X_train = load_boston()['data']
feature_names = load_boston()['feature_names']
# normalizes the data
scaler = StandardScaler().fit(X_train )
X_train = scaler.transform(X_train)


#### Generator Setup
Generator is a feed forward neural network with one hidden layer.  The output layer has the same number of nodes as the real data has columns.  Keep in mind this will not be trained on it's own, but will be stacked with the discriminator

+ n_real_inputs : int number of columns on the input data
+ n_hidden_nodes : int number of nodes in the generator's hidden layer
+ n_fake_inputs : int size of the array of random data used as input to the generator

Array of Noise In -> Array of Fake Data Out

In [3]:
n_real_inputs = 13
n_hidden_nodes = 5
n_fake_inputs = 10

# create a function that returns agenerator
def create_generator(n_real_inputs, n_hidden_nodes, n_fake_inputs):
    inputs = Input(shape=[n_fake_inputs])
    dense_layer0 = Dense(n_hidden_nodes, kernel_initializer='glorot_normal')(inputs)
    batch_norm_layer0 = BatchNormalization()(dense_layer0)
    activation_layer0 = Activation('relu')(batch_norm_layer0)
    dense_layer1 =  Dense(n_real_inputs, kernel_initializer='glorot_normal')(activation_layer0) 
    outputs = Activation('sigmoid')(dense_layer1)
    generator = Model(inputs, outputs)
    generator.compile(loss='categorical_crossentropy', optimizer='adam')
    return generator

generator = create_generator(n_real_inputs = 13, n_hidden_nodes = 5, n_fake_inputs = 10)
generator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
batch_normalization_1 (Batch (None, 5)                 20        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)                78        
_________________________________________________________________
activation_2 (Activation)    (None, 13)                0         
Total params: 153
Trainable params: 143
Non-trainable params: 10
_________________________________________________________________


In [4]:
# rns a test of the generator
test = np.random.normal(size=(2, 10))
generator.predict(test)

array([[0.56643915, 0.49242553, 0.5576671 , 0.6748881 , 0.6383636 ,
        0.4239404 , 0.43823797, 0.31251425, 0.54140085, 0.68441135,
        0.196712  , 0.21001649, 0.49259657],
       [0.5253104 , 0.5042248 , 0.5017169 , 0.54026073, 0.5263095 ,
        0.48517618, 0.49078184, 0.4439832 , 0.50967795, 0.54273593,
        0.42076752, 0.41966155, 0.497997  ]], dtype=float32)

#### Discriminator Setup
The discriminator is a feedforeward nerual network with one hidden layer and drop out

+ n_real_inputs : int number of columns on the input data
+ n_hidden_nodes : int number of nodes in the generator's hidden layer
+ dropout_rate float (probablity at which nodes of the hidden layer randomly are set to zero)

Array of Real and Fake Data,  Arrray [0,1] labels in -> Probality of real or fake out



In [5]:
def create_discriminator(n_real_inputs, n_hidden_nodes, dropout_rate):
    inputs = Input(shape=[n_real_inputs])
    dense_layer0 = Dense(n_hidden_nodes, kernel_initializer='glorot_normal')(inputs)
    activation_layer0 = Activation('relu')(dense_layer0)
    drop0 = Dropout(dropout_rate)(activation_layer0)
    dense_layer1 =  Dense(2, kernel_initializer='glorot_normal')(drop0) 
    outputs = Activation('softmax')(dense_layer1)
    discriminator  = Model(inputs, outputs)
    discriminator.compile(loss='sparse_categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
    return discriminator
discriminator = create_discriminator(n_real_inputs, n_hidden_nodes=5,dropout_rate=.1 )
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 70        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 12        
_________________________________________________________________
activation_4 (Activation)    (None, 2)                 0         
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________


#### Model Stacking 
generator and discriminator are stacked to create one model
+ In the model the Discriminator Model is frozen so that error can backpropigate to the generator
+ The array of noise is paried with an array of [1]s.  Since the discrinator is learning sort out fakes, will force the generator in improve it's ability to make them

Array of Noise, Arrray [1] labels in -> Conversion to Fake Data -> Probality of real or fake out

In [6]:
# Create a stacked version of the model with the generator wieghts frozen
def create_stacked_model(generator, discriminator):
    stacked_model = Sequential()
    generator.trainable = False
    stacked_model.add(generator)
    stacked_model.add(discriminator)
    stacked_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return stacked_model

stacked_model = create_stacked_model(generator, discriminator)
stacked_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 13)                153       
_________________________________________________________________
model_2 (Model)              (None, 2)                 82        
Total params: 235
Trainable params: 82
Non-trainable params: 153
_________________________________________________________________


#### Create Training Data Function
This function takes real data and a generator, and returns array of real data and an array of fake data.  The real data is randomly select to get the batch size correcly.  Each batch is half real and half fake data

+ X_train: real data input
+ generator Keras Model
+ batch: int size of the arrays return (num_rows)


real_data, generator in -> real_data, fake_data

In [7]:
# define the training functions
def create_training_data(X_train, generator, batch=10):
    n_obs, n_inputs = X_train.shape
    half_batch = int(batch/2)
    n_fake_inputs = generator.get_input_shape_at(0)[1]
    # randomly sample real data 
    index =  np.random.randint(0, n_obs, half_batch)
    real_data = X_train[index,:]
    # generate fake data 
    noise = np.random.normal(0, 1, (half_batch, n_fake_inputs))
    fake_data =  generator.predict(noise)
    return real_data, fake_data 


#### Primary Training Function
+ X_train: training array
+ generator: Keras Generator Model
+ stacked_model:  Keras Generator stacked with Discriminator (discriminator weights frozen)
+ epochs: int number of batches  
+ batch: int number of rows per batch, (half real and half fake data)

In [8]:
def training(X_train, generator, discriminator, epochs=10, batch=10):
    # create the stacked model
    stacked_model = create_stacked_model(generator, discriminator)
    # get the input data shape
    n_obs, n_inputs = X_train.shape
    half_batch = int(batch/2)
    n_fake_inputs = generator.get_input_shape_at(0)[1]
    for e in range(epochs):
        real_data, fake_data = create_training_data(X_train, generator, batch)
        combined_data = np.concatenate((real_data, fake_data))
        real_labels = np.ones(half_batch)
        fake_labels = np.zeros(half_batch)
        combined_labels = np.append(real_labels, fake_labels)
        
        # discriminator loss 
        disc_loss = discriminator.train_on_batch(combined_data ,combined_labels)
        
        # create noise and labels of 1,
        noise =  np.random.normal(0, 1, (batch, n_fake_inputs))
        y_mislabled =  np.ones((batch))
        
        # genertor loss 
        gen_loss = stacked_model.train_on_batch(x=noise,y=y_mislabled)
        logger.info('epoch:{0}/{1}, disc_loss: {2}, gen_loss:{3}'.format(e, epochs, disc_loss, gen_loss))

    generator = stacked_model.layers[0]
    discriminator = stacked_model.layers[1]
    return generator, discriminator

generator, discriminator  = training(X_train, generator, discriminator,  epochs=10, batch=1000)

2019-10-03 14:27:08,620 - GANs - INFO - epoch:0/10, disc_loss: [0.61967427, 0.744], gen_loss:1.3066082000732422
2019-10-03 14:27:08,656 - GANs - INFO - epoch:1/10, disc_loss: [0.621907, 0.76], gen_loss:1.3072524070739746
2019-10-03 14:27:08,693 - GANs - INFO - epoch:2/10, disc_loss: [0.6206468, 0.756], gen_loss:1.287769079208374
2019-10-03 14:27:08,729 - GANs - INFO - epoch:3/10, disc_loss: [0.5974836, 0.763], gen_loss:1.2801730632781982
2019-10-03 14:27:08,765 - GANs - INFO - epoch:4/10, disc_loss: [0.6123271, 0.75], gen_loss:1.2691081762313843
2019-10-03 14:27:08,800 - GANs - INFO - epoch:5/10, disc_loss: [0.61043376, 0.754], gen_loss:1.2368003129959106
2019-10-03 14:27:08,836 - GANs - INFO - epoch:6/10, disc_loss: [0.5983659, 0.754], gen_loss:1.2508504390716553
2019-10-03 14:27:08,873 - GANs - INFO - epoch:7/10, disc_loss: [0.60950464, 0.756], gen_loss:1.2318276166915894
2019-10-03 14:27:08,909 - GANs - INFO - epoch:8/10, disc_loss: [0.62912416, 0.742], gen_loss:1.2034859657287598
2

#### Generating Fake Data 
using random normals to generate face data, this essentialy reverses the scaler to get the data back on the original scale

In [9]:
## make some fake data
def make_fake_data(batch, generator, scaler):
    n_fake_inputs =  generator.get_weights()[0].shape[0]
    noise =  np.random.normal(0, 1, (batch, n_fake_inputs))
    fake_data = generator.predict(noise)
    fake_data_unscaled = scaler.inverse_transform(fake_data)
    return fake_data_unscaled 
print('FAKE :')
pd.DataFrame(make_fake_data(batch=4, generator=generator, scaler=scaler), columns=feature_names)    


FAKE :


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,7.154866,21.322029,15.625684,0.220020,0.627697,6.590303,80.261192,4.942539,14.035790,509.117096,19.204794,394.913300,16.223076
1,5.409275,23.449846,17.518940,0.322013,0.603792,6.390985,91.540504,3.893902,10.295435,576.231201,18.457268,373.290985,15.396666
2,5.341480,20.742504,17.268656,0.311714,0.617764,6.466686,86.482071,4.436302,11.241587,571.005981,18.502550,396.316742,15.987784
3,7.910044,23.013334,14.563564,0.196041,0.612577,6.635596,82.635414,4.846857,13.898733,492.422394,19.536936,402.276337,16.220064


In [10]:
print('REAL :')
test_data = load_boston()['data'] 
pd.DataFrame(test_data[0:4,:], columns=feature_names)

REAL :


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94


#### Looking for Fake Data or Outliers
The discriminator function can be used to detect outliers or look for fake data 



In [11]:
def is_fake(x, dicriminator, scaler):
    x_scaled = scaler.transform(x)
    prob =  dicriminator.predict(x_scaled)
    return prob
 
pd.DataFrame(is_fake(test_data, discriminator, scaler), columns=['fake', 'real']).head()

,fake,real
0,0.547792,0.452208
1,0.184778,0.815222
2,0.334692,0.665308
3,0.217157,0.782842
4,0.189465,0.810535


#### Saving the Models
If you want to save the model
and load it later for use 

In [12]:
path = "_model.h5"
model = generator
# serialize weights to HDF5
model.save(path,  overwrite=True)
print("Saved model to {}".format(path))

Saved model to _model.h5


#### loading Model

In [13]:
# load model
model_loaded = load_model(path)
# summarize model.
model_loaded.summary()
make_fake_data(2, model_loaded, scaler)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
batch_normalization_1 (Batch (None, 5)                 20        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)                78        
_________________________________________________________________
activation_2 (Activation)    (None, 13)                0         
Total params: 153
Trainable params: 143
Non-trainable params: 10
_________________________________________________________________


array([[7.5817661e+00, 2.2790003e+01, 1.4636750e+01, 2.2389206e-01,
        6.1003739e-01, 6.6523571e+00, 8.4787682e+01, 4.7551684e+00,
        1.3559386e+01, 5.0940768e+02, 1.9177650e+01, 4.0985922e+02,
        1.6514050e+01],
       [7.5191550e+00, 2.2815746e+01, 1.4599019e+01, 1.9653280e-01,
        6.1633396e-01, 6.6510787e+00, 8.2009132e+01, 4.9424906e+00,
        1.4604495e+01, 4.8578214e+02, 1.9499296e+01, 4.0600519e+02,
        1.6175621e+01]], dtype=float32)

In [16]:
#cleanup
os.remove(path)
